# Plotting Hair Cell Orientation in Polar Plots

Prepared by: Abel P. David (Northwestern University)

Date: 11/6/2025

Prepared for: Ippei Kishimoto and Alan Cheng (Stanford University)

## Install Dependencies

In [ ]:
%pip install matplotlib numpy scipy pandas openpyxl

## Load Libraries

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from collections import defaultdict
import pandas as pd

## Change Figure Settings

In [ ]:
%matplotlib inline
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = "Arial"
mpl.rcParams['figure.dpi'] = 300

## Functions

In [ ]:
def deg2rad(degrees):
    """Convert degrees to radians."""
    return np.deg2rad(degrees)

def rad2deg(radians):
    """Convert radians to degrees."""
    return np.rad2deg(radians)

def mean_resultant_length(angles, deg=False):
    """
    Compute the mean resultant length (R), a measure of concentration.
    Args:
        angles : array-like
        deg    : bool, if True, input in degrees
    Returns:
        R : float, between 0 and 1
    """
    if deg:
        angles = np.deg2rad(angles)
    sin_mean = np.mean(np.sin(angles))
    cos_mean = np.mean(np.cos(angles))
    R = np.sqrt(sin_mean**2 + cos_mean**2)
    return R

def circular_mean(angles, deg=False):
    """
    Compute the circular mean of angles.
    Args:
        angles : array-like, list of angles
        deg    : bool, if True, input/output in degrees
    Returns:
        mean_angle : float, circular mean
    """
    if deg:
        angles = np.deg2rad(angles)
    mean_angle = np.arctan2(np.mean(np.sin(angles)), np.mean(np.cos(angles)))
    if deg:
        mean_angle = np.rad2deg(mean_angle) % 360
    return mean_angle

def circular_variance(angles, deg=False):
    """
    Compute the circular variance.
    Args:
        angles : array-like
        deg    : bool, if True, input in degrees
    Returns:
        variance : float
    """
    R = mean_resultant_length(angles, deg=deg)
    return 1 - R

def circular_std(angles, deg=False):
    """
    Compute the circular standard deviation.
    Args:
        angles : array-like
        deg    : bool, if True, input/output in degrees
    Returns:
        std : float
    """
    R = mean_resultant_length(angles, deg=deg)
    std = np.sqrt(-2 * np.log(R))
    if deg:
        std = np.rad2deg(std)
    return std

## Example usage:
angles_deg = [350, 10, 20]

print("Circular mean:", circular_mean(angles_deg, deg=True))
print("Mean resultant length (R):", mean_resultant_length(angles_deg, deg=True))
print("Variance:", circular_variance(angles_deg, deg=True))
print("Std Dev:", circular_std(angles_deg, deg=True))

In [ ]:
def compute_circular_stats(df, output_csv):
    """
    Compute circular statistics (mean, std, variance, mean resultant length)
    for each row in a DataFrame, then drop unnecessary columns and save to CSV.

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing a column 'values' with numeric arrays (angles in degrees).
    output_csv : str
        Output filename for the CSV.

    Returns
    -------
    df_out : pandas.DataFrame
        Processed dataframe with circular statistics columns added and specified columns removed.
    """
    # Loop through rows to calculate circular statistics
    for index, row in df.iterrows():
        values = row["values"]
        avg = circular_mean(values, deg=True)
        sd = circular_std(values, deg=True)
        R = mean_resultant_length(values, deg=True)
        circular_variance = 1 - R
        n = len(values)

        df.at[index, "circular_mean"] = avg
        df.at[index, "circular_std"] = sd
        df.at[index, "mean_resultant_length"] = R
        df.at[index, "circular_variance"] = circular_variance
        df.at[index, "n_measurements"] = n

    # Drop unneeded columns if they exist
    drop_cols = ["title", "values", "average", "sd", "values"]
    df_out = df.drop(columns=[c for c in drop_cols if c in df.columns])

    # Save to CSV
    df_out.to_csv(output_csv, index=False)

    return df_out


In [ ]:
def polar_direction_plot(
    data,
    title="Polar Plot",
    bin_deg=5,
    figsize=(5, 5),
    avg_color="tab:blue",
    bar_color="tab:green",
    sd_color="tab:red",
    grid_color="#BEBEBE",
    save_path=None,
    show=True,
):
    """
    Make a polar 'rose' histogram with an average direction line and an SD arc.

    Parameters
    ----------
    data : sequence of float
        Angles in degrees (can be negative; range -180..180 is fine).
    avg_deg : float
        Mean direction in degrees (0 at North, increases clockwise).
    sd_deg : float
        Circular standard deviation (in degrees).
    title : str
        Title for the plot.
    bin_deg : int
        Bin width in degrees (default 5). Must divide 360 evenly.
    figsize : tuple
        Figure size in inches.
    avg_color, bar_color, sd_color : str
        Matplotlib color specs for the average line, bars, and SD arc.
    grid_color : str
        Grid/guide color for background lines/circles.
    save_path : str or None
        If provided, saves the figure to this path.
    show : bool
        If True, shows the figure.

    Returns
    -------
    fig, (ax1, ax2)
        Matplotlib figure and polar axes (ax1 for drawing, ax2 for annotations).
    """
    # --- Validate binning ---
    if 360 % bin_deg != 0:
        raise ValueError("bin_deg must evenly divide 360.")

    # Normalize inputs
    data = np.asarray(data, dtype=float)

    # Build bin centers from -180 to +180 (inclusive) spaced by bin_deg
    # Indices will run from -N//2 to +N//2, matching your original -36..36 for bin_deg=5
    n_bins = 360 // bin_deg
    half_bins = n_bins // 2
    bin_indices = np.arange(-half_bins, half_bins + 1)  # inclusive centers
    # Convert an angle to its nearest bin index (round to nearest bin center)
    def angle_to_index(a):
        # Shift so that 0 aligns to center of bin [−bin_deg/2, +bin_deg/2]
        return int(np.round(a / bin_deg))

    # Collect values per bin index
    buckets = defaultdict(list)
    for val in data:
        # Clamp to range [-180,180]; data outside gets wrapped
        # Wrap to (-180,180], then map to index space
        v = ((val + 180) % 360) - 180
        idx = angle_to_index(v)
        # Keep indices within [-half_bins, +half_bins]
        idx = max(-half_bins, min(half_bins, idx))
        buckets[idx].append(v)

    # --- Figure / axes ---
    fig = plt.figure(figsize=figsize)
    ax1 = plt.subplot(111, polar=True)
    plt.grid(color='white')
    ax2 = plt.subplot(111, polar=True)

    # Axes styling
    ax1.axis("off")
    ax1.set_theta_zero_location('N')
    ax2.set_theta_zero_location('N')
    ax2.set_title(title, ax2.set_rlabel_position(270), fontsize=10, zorder=12)
    ax1.xaxis.grid(True, color=grid_color)
    ax1.tick_params(axis='y', colors='white')

    # Determine maximum radius (the tallest bar)
    r = max((len(buckets[i]) for i in bin_indices), default=0)


    avg_deg = circular_mean(data, deg=True)
    sd_deg = circular_std(data, deg=True)

    # --- Average direction line ---
    theta_avg = deg2rad(avg_deg)
    ax1.vlines(theta_avg, 0, r, colors=avg_color, zorder=7, linewidth=2)

    # --- Bars ---
    width = np.deg2rad(bin_deg)
    for i in bin_indices:
        height = len(buckets[i])
        theta_center = np.deg2rad(i * bin_deg)
        ax1.bar(
            theta_center, height,
            width=width,
            color=bar_color,
            edgecolor=None,
            zorder=6
        )

    # --- SD arc (centered on avg, radius at half of max bar height) ---
    
    if r > 0 and sd_deg is not None and np.isfinite(sd_deg):
        radius = 0.5 * r
        start = avg_deg + sd_deg
        end   = avg_deg - sd_deg 
        x = np.linspace(np.deg2rad(start), np.deg2rad(end), 500)
        y = interp1d([np.deg2rad(start), np.deg2rad(end)], [radius, radius])(x)
        ax1.plot(x, y, zorder=8, color=sd_color, linewidth=2)

    # --- Annotations ---
    ax2.annotate(
        f'n={len(data)}',
        xy=(np.deg2rad(90), r),
        xytext=(0.7, 0.1),
        textcoords='figure fraction',
        ha='left', va='bottom', fontsize=8
    )

    # --- Concentric circles (quartiles of max radius) ---
    for rad, lw, ec in [(r/4, 0.5, grid_color), (r/2, 1.0, "#767676"),
                        (3*r/4, 0.5, grid_color), (r, 1.0, "black")]:
        circ = plt.Circle((0, 0), rad, transform=ax1.transData._b, fill=False,
                          edgecolor=ec, linewidth=lw, zorder=2)
        plt.gca().add_artist(circ)

    # --- Background radial spokes every 45° ---
    for deg in [0, 45, 90, 135, 180, 225, 270, 315]:
        ax1.vlines(np.deg2rad(deg), 0, r, colors=grid_color, zorder=1, linewidth=1)

    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    if show:
        plt.show()

    return fig, (ax1, ax2)


## Extract All Excel Filenames

In [ ]:
# get list of excel files in the data directory and creat a list of their filenames
import os   
data_dir = "./data"
excel_files = [f for f in os.listdir(data_dir) if f.endswith('.xlsx')]

print("Excel files found:", excel_files)

## Create Polar Statistics and Plots

In [ ]:
# for loop through the excel files and read them into dataframes
for excel_file in excel_files:
    file_path = os.path.join(data_dir, excel_file)
    df = pd.read_excel(file_path, engine='openpyxl')
    print(f"Processing file: {excel_file}")
    df["values"] = df["values"].apply(
        lambda x: np.fromstring(str(x), sep=",")
    )
    # create a title combining condition and location
    df["title"] = df["condition"] + " " + df["location"]

    # calculate circular statistics and save to new csv
    results_dir = "./results/"
    output_csv = os.path.join(results_dir, excel_file.replace('.xlsx', '_circular_stats.csv'))
    df_stats = compute_circular_stats(df, output_csv)
    print(f"Circular statistics saved to: {output_csv}")

    # create polar plots for each row in the dataframe saved into a subdirectory of results, create the subdirectory if it doesn't exist
    results_dir = "./results/" + excel_file.replace('.xlsx', '')
    os.makedirs(results_dir, exist_ok=True)
    for index, row in df.iterrows():
        angles = row["values"]
        title = row["title"]
        avg_deg = row["circular_mean"]
        sd_deg = row["circular_std"]
        plot_filename = f"{results_dir}/{excel_file.replace('.xlsx', '')}_polar_plot_{title}.pdf"
        polar_direction_plot(
            figsize=(3,3),
            data=angles,
            title=title,
            avg_color=plt.get_cmap('tab20')(4), # dark green
            bar_color=plt.get_cmap('tab20')(5), # light green
            sd_color=plt.get_cmap('tab20')(6), # red
            save_path=plot_filename,
            show=False
        )
        print(f"Polar plot saved to: {plot_filename}")
